In [2]:
import torch
from collections import defaultdict
import numpy as np
import mcubes
import os
import trimesh

from models.rendering import *
from models.nerf import *
from torchsummary import summary
from datasets import dataset_dict
from utils import load_ckpt, extract_model_state_dict, load_ckpt_other

# Load model and data

In [3]:
# Change here #
img_wh = (480, 360) # full resolution of the input images
dataset_name = 'llff' # blender or llff (own data)
scene_name = '250_most_depth' # whatever you want
root_dir = '/home/bowmonk/bladenerf//Hierarchical-Localization/outputs/larger_reconstruction/larger_reconstruction_250/' # the folder containing data
ckpt_path = '/home/bowmonk/bladenerf/nerf_pl/ckpts/large_80/epoch=29.ckpt' # the model path
load_path ='/home/bowmonk/bladenerf/DSNeRF/logs/release/250_frames_tensorboard_most_depth/300000.tar'
###############

kwargs = {'root_dir': root_dir,
          'img_wh': img_wh}
if dataset_name == 'llff':
    kwargs['spheric_poses'] = True
    kwargs['split'] = 'test'
else:
    kwargs['split'] = 'train'
    
chunk = 1024*16
dataset = dataset_dict[dataset_name](**kwargs)

embedding_xyz = Embedding(10)
embedding_dir = Embedding(4)

nerf_fine = NeRF()
nerf_fine_other = NeRF(in_channels_xyz=63, in_channels_dir=27)
load_ckpt(nerf_fine, ckpt_path, model_name='nerf_fine')
checkpoint = extract_model_state_dict(load_path, model_name='nerf_fine')
# load_ckpt(nerf_fine_other, ckpt_path, model_name='nerf_fine')

nerf, nerf_fine = load_ckpt_other()

nerf_fine.cuda().eval();


Cameras 5 {'id': 1, 'model': 'SIMPLE_RADIAL', 'width': 479, 'height': 358, 'params': [278.12527177107853, 239.5, 17.0, -0.13445126237284866]}
<datasets.llff.LLFFDataset object at 0x7f79fb2484d0> yoooooooooooooo
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh2
huh2
huh2
{}
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
huh2
huh3
Found ckpts ['../DSNeRF/logs/release/250_frame

# Search for tight bounds of the object (trial and error!)

In [4]:
### Te these parameters until the whole object lies tightly in range with little noise ###
N = 512 # controls the resolution, set this number small here because we're only finding
        # good ranges here, not yet for mesh reconstruction; we can set this number high
        # when it comes to final reconstruction.
xmin, xmax = -12, 12 # left/right range
ymin, ymax = -12, 12 # forward/backward range
zmin, zmax = -12, 12 # up/down range
## Attention! the ranges MUST have the same length!
sigma_threshold = 10. # controls the noise (lower=maybe more noise; higher=some mesh might be missing)
############################################################################################

x = np.linspace(xmin, xmax, N)
y = np.linspace(ymin, ymax, N)
z = np.linspace(zmin, zmax, N)

xyz_ = torch.FloatTensor(np.stack(np.meshgrid(x, y, z), -1).reshape(-1, 3)).cuda()
dir_ = torch.zeros_like(xyz_).cuda()

with torch.no_grad():
    B = xyz_.shape[0]
    out_chunks = []
    other_out_chunks = []
    for i in range(0, B, chunk):
        xyz_embedded = embedding_xyz(xyz_[i:i+chunk]) # (N, embed_xyz_channels)
        dir_embedded = embedding_dir(dir_[i:i+chunk]) # (N, embed_dir_channels)
        xyzdir_embedded = torch.cat([xyz_embedded, dir_embedded], 1)
        out_chunks += [nerf_fine(xyzdir_embedded)]
    rgbsigma = torch.cat(out_chunks, 0)

    
sigma = rgbsigma[:, -1].cpu().numpy()
sigma = np.maximum(sigma, 0)
sigma = sigma.reshape(N, N, N)

# The below lines are for visualization, COMMENT OUT once you find the best range and increase N!
# vertices, triangles = mcubes.marching_cubes(sigma, sigma_threshold)
# mesh = trimesh.Trimesh(vertices/N, triangles)
# mesh.show()

In [5]:
# # You can already export "colorless" mesh if you don't need color
mcubes.export_mesh(vertices, triangles, f"{scene_name}.dae")

/home/bowmonk/anaconda3/envs/nerf_pl/lib/python3.7/site-packages/collada/geometry.py:232: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  if src.xmlnode not in meshnode.getchildren():
/home/bowmonk/anaconda3/envs/nerf_pl/lib/python3.7/site-packages/collada/geometry.py:294: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  if prim.xmlnode not in meshnode.getchildren():
/home/bowmonk/anaconda3/envs/nerf_pl/lib/python3.7/site-packages/collada/geometry.py:299: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  for child in meshnode.getchildren():


# Generate .vol file for volume rendering in Unity

In [5]:
assert N==512, \
    'Please set N to 512 in the two above cell! Remember to comment out the visualization code (last 3 lines)!'


a = 1-np.exp(-(xmax-xmin)/N*sigma)
a = a.flatten()

rgb = (rgbsigma[:, :3].cpu().numpy()*255).astype(np.uint32)

i = np.where(a>0)[0] # valid indices (alpha>0)

rgb = rgb[i]
a = a[i]
s = rgb.dot(np.array([1<<24, 1<<16, 1<<8])) + (a*255).astype(np.uint32)
res = np.stack([i, s], -1).astype(np.uint32).flatten()
with open(f'{scene_name}.vol', 'wb') as f:
    f.write(res.tobytes())

[0.99999994 1.         0.99999964 ... 0.         0.         0.        ]
tensor([[ 3.5015e-04,  3.7227e-04,  5.3494e-04,  3.5870e+02],
        [ 1.8565e-04,  1.9926e-04,  2.9594e-04,  3.7942e+02],
        [ 1.0354e-04,  1.0992e-04,  1.6602e-04,  3.1742e+02],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -2.0457e+03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -2.0559e+03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -2.0688e+03]], device='cuda:0')
[[0 0 0]
 [0 0 0]
 [0 0 0]
 ...
 [0 0 0]
 [0 0 0]
 [0 0 0]]


# Extract colored mesh

Once you find the best range, now **RESTART** the notebook, and copy the configs to the following cell
and execute it.

In [6]:
# Copy the variables you have above here! ####
img_wh = (480, 360) # full resolution of the input images
dataset_name = 'llff' # blender or llff (own data)
scene_name = '250_frames_tensorboard_no_depth' # whatever you want
root_dir = '/home/bowmonk/bladenerf//Hierarchical-Localization/outputs/larger_reconstruction/larger_reconstruction_250/' # the folder containing data
ckpt_path = '/home/bowmonk/bladenerf/nerf_pl/ckpts/large_80/epoch=29.ckpt' # the model path
load_path ='/home/bowmonk/bladenerf/DSNeRF/logs/release/250_frames_tensorboard_no_depth/300000.tar'
###############
N = 512 # controls the resolution, set this number small here because we're only finding
        # good ranges here, not yet for mesh reconstruction; we can set this number high
        # when it comes to final reconstruction.
xmin, xmax = -13, 13 # left/right range
ymin, ymax = -13, 13 # forward/backward range
zmin, zmax = -13, 13 # up/down range
## Attention! the ranges MUST have the same length!
sigma_threshold = 5. # controls the noise (lower=maybe more noise; higher=some mesh might be missing)
## Attention! the ranges MUST have the same length!
# sigma_threshold = 10. # controls the noise (lower=maybe more noise; higher=some mesh might be missing)
###############################################

import os
os.environ['ROOT_DIR'] = root_dir
os.environ['DATASET_NAME'] = dataset_name
os.environ['SCENE_NAME'] = scene_name
os.environ['IMG_SIZE'] = f"{img_wh[0]} {img_wh[1]}"
os.environ['CKPT_PATH'] = ckpt_path
os.environ['N_GRID'] = "128" # final resolution. You can set this number high to preserve more details
os.environ['X_RANGE'] = f"{xmin} {xmax}"
os.environ['Y_RANGE'] = f"{ymin} {ymax}"
os.environ['Z_RANGE'] = f"{zmin} {zmax}"
os.environ['SIGMA_THRESHOLD'] = str(sigma_threshold)
os.environ['OCC_THRESHOLD'] = "0.1" # probably doesn't require tuning. If you find the color is not close
                                    # to real, try to set this number smaller (the effect of this number
                                    # is explained in my youtube video)
os.environ['USE_VERTEX_NORMAL'] = 'True'
!python extract_color_mesh.py \
    --root_dir $ROOT_DIR \
    --dataset_name $DATASET_NAME \
    --scene_name $SCENE_NAME \
    --img_wh $IMG_SIZE \
    --ckpt_path $CKPT_PATH \
    --N_grid $N_GRID \
    --x_range $X_RANGE \
    --y_range $Y_RANGE \
    --z_range $Z_RANGE \
    --sigma_threshold $SIGMA_THRESHOLD \
    --occ_threshold $OCC_THRESHOLD \
    --use_vertex_normal

Cameras 5 {'id': 1, 'model': 'SIMPLE_RADIAL', 'width': 479, 'height': 358, 'params': [278.12527177107853, 239.5, 17.0, -0.13445126237284866]}
<datasets.llff.LLFFDataset object at 0x7fd0af4e70d0> yoooooooooooooo
Found ckpts ['../DSNeRF/logs/release/250_frames_tensorboard_most_depth/299000.tar', '../DSNeRF/logs/release/250_frames_tensorboard_most_depth/300000.tar']
Reloading from ../DSNeRF/logs/release/250_frames_tensorboard_most_depth/300000.tar
Predicting occupancy ...
100%|███████████████████████████████████████████| 64/64 [00:00<00:00, 76.93it/s]
Extracting mesh ...
Removing noise ...
Mesh has 0.35 M vertices and 0.72 M faces.
True
Found ckpts ['../DSNeRF/logs/release/250_frames_tensorboard_most_depth/299000.tar', '../DSNeRF/logs/release/250_frames_tensorboard_most_depth/300000.tar']
Reloading from ../DSNeRF/logs/release/250_frames_tensorboard_most_depth/300000.tar
Done!
